# Product Analysis

Use jupyter notebook to conduct interactive data analysis.

Also output crucial files for data visualization on web.

In [2]:
import numpy as np
import pandas as pd
import json
import math
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# json encoder for numpy
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return None if math.isnan(obj) else float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
# Define a function to remove HTML tags from a string
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped = soup.get_text(separator=" ")
    return stripped

COMMENT_SAMPLE_N = 10
PRODUCT_SAMPLE_N = 5

customer_csv_path = '../data/csv/aug_customers.csv'
product_csv_path = '../data/csv/aug_products.csv'
comment_csv_path = '../data/csv/aug_comments.csv'

output_base_dir = '../stats/product'

In [33]:
# parse selected comments to json form
# and join info from products & customers
def parse_joined_comments_to_json(selected_comment_df):
    comments = []
    for idx, comment_ds in selected_comment_df.iterrows():
        comment = {
            'summary': None if isinstance(comment_ds['summary'], float) and math.isnan(comment_ds['summary']) else comment_ds['summary'],
            'review': comment_ds['review'],
            'timestamp': comment_ds['timestamp'],
            'rating': comment_ds['overall'],
            'sentiment': comment_ds['sentiment'],
            'overall': comment_ds['sentimentWithRating'],
            'wordCount': comment_ds['wordCount'],
        }
        customer_ds = customer_df[customer_df['customerId']==comment_ds['customerId']].iloc[0]
        product_ds = product_df[product_df['asin']==comment_ds['asin']].iloc[0]
        comment['customer'] = {
            'customerId': customer_ds['customerId'],
            'name': customer_ds['name'],
            'cluster': customer_ds['cluster'],
        }
        comment['product'] = {
            'asin': product_ds['asin'],
            'title': product_ds['title'],
            'brand': None if isinstance(product_ds['brand'], float) and math.isnan(product_ds['brand']) else product_ds['brand'],
            # 'imageUrl': product_ds['imageUrl'],
            'imageUrl': None if isinstance(product_ds['imageUrl'], float) and math.isnan(product_ds['imageUrl']) else product_ds['imageUrl'],
            'price': product_ds['price'],
            'categories': product_ds['categories'].split(','),
            'avgRating': product_ds['avgOverall'],
            'avgSentiment': product_ds['avgSentiment'],
            'avgOverall': product_ds['avgSentimentWithRating'],
            'reviewCount': product_ds['reviewCount'],
        }
        comments.append(comment)
    return comments

## Load Data

In [4]:
product_df = pd.read_csv(product_csv_path)
total_count = len(product_df)

print(f'Number of products: {total_count}')

/tmp/ipykernel_55801/201748605.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  product_df = pd.read_csv(product_csv_path)


Number of products: 181839


In [13]:
product_df = product_df[product_df['reviewCount'] > 0]
total_count = len(product_df)

print(f'Number of products with comments: {total_count}')


Number of products with comments: 180424


In [5]:
comment_df = pd.read_csv(comment_csv_path)
comment_df.head()

,overall,customerId,asin,summary,review,timestamp,sentiment,sentimentWithRating,wordCount
0,1.0,A00013803RVZPCZKTT9U,B003ZTNT2Y,One Star,Crap!!!!,1485129600,-1.0000,1.00000,1
1,5.0,A0001392IVCRENBEIEYS,6302409365,I'm so glad I watched this,This made my night. I'm so glad I watched this.,1488240000,0.5000,8.75000,9
2,5.0,A0001598OL7FAN6XNMK9,B00BMRTPEM,its riveting. i have to keep myself controlled...,its riveting. i have to keep myself controlled...,1478649600,0.2500,8.12500,22
3,4.0,A0001598OL7FAN6XNMK9,B00IV3FLO8,Four Stars,Great action. Good twists. It's sexy it's hot!,1475625600,0.4825,7.70625,8
4,5.0,A0001598OL7FAN6XNMK9,B00OGL6S64,its riveting. i have to keep myself controlled...,its riveting. i have to keep myself controlled...,1478649600,0.2500,8.12500,22


In [6]:
customer_df = pd.read_csv(customer_csv_path)

customer_df.head()

,customerId,name,cluster,distanceToCenter,vector2dX,vector2dY
0,A3478QRKQDOPQ2,jacki,2,13.761846,6.972486,13.825474
1,A2VHSG6TZHU1OB,Ken P,3,33.170107,15.408235,-2.924636
2,A23EJWOW1TLENE,Reina Berumen,0,8.186462,-1.312076,5.836580
3,A1KM9FNEJ8Q171,N Coyle,0,12.849782,-5.788060,5.276560
4,A38LY2SSHVHRYB,Jodie Vesely,0,12.660837,-0.643088,-1.748742


In [14]:
product_df.head()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
51,0253030471,Cuba: The Forgotten Revolution,NaN,<P>Cuba: The Forgotten Revolution tells the vi...,NaN,170494,24.95,Movies,5.0,0.350000,8.375000,1
44,0075723840,Direct Instruction Professional Development Vi...,NaN,Set of Six VHS cassettes,https://images-na.ssl-images-amazon.com/images...,1040896,-1.00,TV,5.0,0.550000,8.875000,1
5,000073991X,The Power of the Cross Joseph Prince,Joseph Prince,Have failures in your life caused you to feel ...,NaN,444474,-1.00,"Genre for Featured Categories,Exercise & Fitness",5.0,0.225000,8.062500,1
181791,B01HFFZNCW,As We Speak - Lyric Video,Greater Vision,1. Put Out The Fire 2. We Can't Tell It All 3....,NaN,269649,13.98,"Genre for Featured Categories,Music Videos & C...",5.0,0.262338,8.155844,1
181792,B01HFH0NI4,The X-Files: The Event Series [Blu-ray],David Duchovny,"Thirteen years after the original series run, ...",NaN,370140,-1.00,"Science Fiction & Fantasy,Science Fiction,Alie...",5.0,0.492312,8.730781,1


In [15]:
product_df.describe()

,price,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
count,180424.000000,180424.000000,180424.000000,180424.000000,180424.000000
mean,10.482098,4.055638,0.270196,7.231127,47.411026
std,21.633057,0.977952,0.201660,1.319454,309.753167
min,-1.000000,1.000000,-1.000000,1.000000,1.000000
25%,-1.000000,3.636364,0.156517,6.621340,2.000000
50%,5.980000,4.333333,0.269341,7.518625,4.000000
75%,14.990000,4.833333,0.376019,8.102461,17.000000
max,999.990000,5.000000,1.000000,10.000000,24088.000000


## Product Comment Count

In [16]:
# sort by length
product_df.sort_values(by=['reviewCount'], inplace=True)

product_df.head()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
68865,B000LE4JA2,Exploring Horizons A Rugged Beauty - North Van...,NaN,"From the aerial view to the human perspective,...",NaN,832806,19.99,"Genre for Featured Categories,Special Interests",1.0,0.250000,4.125000,1
68868,B000LE872S,Exploring Horizons Caribbean Soul - Caye Caulk...,NaN,"From the aerial view to the human perspective,...",NaN,604668,19.99,"Genre for Featured Categories,Special Interests",2.0,-0.051786,4.370536,1
143535,B009XF3EOC,Bike-O-Vision Cycling Video- Eastern Sierra Hi...,none,Splashes of fall color highlight dramatic moun...,NaN,431908,22.95,"Blu-ray,Movies",5.0,0.396429,8.491071,1
143581,B009YDW5NO,GOSICK BLU-RAY BOX(4BLU-RAY+CD+BOOKLET),NaN,Japanese Rare Items,https://images-na.ssl-images-amazon.com/images...,807593,-1.00,"Blu-ray,Movies",5.0,0.051852,7.629630,1
143582,B009YEUSF0,DAS B&ouml;se - Phantasm-box 1-4,NaN,Quick Shipping !!! New And Sealed !!! This Dis...,https://images-na.ssl-images-amazon.com/images...,427681,-1.00,"Art House & International,By Original Language...",3.0,0.000000,5.500000,1


In [10]:
product_df.tail()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
178936,B01BHTSIOC,Doctor Thorne - Season 1 2015,NaN,"Tom Hollander (<em>Rev</em>, <em>A Poet In New...",https://images-na.ssl-images-amazon.com/images...,123323,14.88,"Genre for Featured Categories,Romance",4.423445,0.446030,8.038519,20508
145635,B00AQVMZKQ,Downton Abbey - Series 4,Maggie Smith,"NOTICE: Polish Release, cover may contain Poli...",https://images-na.ssl-images-amazon.com/images...,165427,17.74,"Blu-ray,TV",4.829337,0.455143,8.467196,20602
81360,B000WGWQG8,John Adams,Paul Giamatti,John Adams (DVD),https://images-na.ssl-images-amazon.com/images...,49694,38.66,"HBO,All HBO Titles",4.686705,0.438693,8.283437,23106
31289,B00006CXSS,Band Of Brothers (DVD),Damian Lewis,"Based on the bestseller by Stephen E. Ambrose,...",https://images-na.ssl-images-amazon.com/images...,14431,-1.00,"HBO,All HBO Titles",4.913773,0.513329,8.697096,24018
173750,B00YSG2ZPA,Band of Brothers(Elite SC/BD+DCExp12-21),Various,<![CDATA[,https://images-na.ssl-images-amazon.com/images...,503,25.16,"HBO,All HBO Titles",4.913193,0.513080,8.695894,24088


In [18]:
product_review_count_stat = []
thresholds = [1, 10, 100, 1000, 10000]

for idx, thres in enumerate(thresholds):
    if idx < len(thresholds) - 1:
        cat = f'{thres} ~ {thresholds[idx+1]-1} comments'
        subset_df = product_df[product_df["reviewCount"].between(thres, thresholds[idx+1], inclusive='left')]
    else:
        cat = f'>= {thres} comments'
        subset_df = product_df[product_df["reviewCount"] >= thres]
    count = len(subset_df)
    print(f'{count} comments in category {cat}, accounts for {100*count/total_count:.2f}%')
    
    product_review_count_stat.append({
        'category': cat,
        'count': count,
    })
    

118855 comments in category 1 ~ 9 comments, accounts for 65.88%
48412 comments in category 10 ~ 99 comments, accounts for 26.83%
11790 comments in category 100 ~ 999 comments, accounts for 6.53%
1341 comments in category 1000 ~ 9999 comments, accounts for 0.74%
26 comments in category >= 10000 comments, accounts for 0.01%


Another example of 80/20 rule: 65% of products only have less than 10 comments.

In [19]:
with open(os.path.join(output_base_dir, 'comment_count.json'), 'w') as f:
    json.dump(product_review_count_stat, f, cls=NpEncoder)

# Best Products

Select products with at least 10 comments.

In [29]:
# select products with at least 20 comments
product_df = product_df[product_df['reviewCount'] >= 20]

print(f'{len(product_df)} products have at least 20 comments')

41678 products have at least 20 comments


In [40]:
product_df = product_df.sort_values(by=['avgSentimentWithRating'], ascending=False)

In [27]:
product_df.head()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
176284,B015S222SW,Wonderful World - Live In Maastricht,NaN,NTSC/Region 0. Andr's fans have been looking f...,https://images-na.ssl-images-amazon.com/images...,68332,19.38,"Musicals & Performing Arts,Classical",4.967742,0.640273,9.068425,31
169748,B00S64UYOY,Passion Talk Series: The Complete Collection,Louie Giglio,The all new PASSION TALK SERIES: THE COMPLETE ...,NaN,31928,199.95,"Genre for Featured Categories,Faith & Spiritua...",4.964286,0.615060,9.001937,28
164270,B00LHD6K62,The Notebook / The Time Traveler's Wife,Various,<![CDATA[,https://images-na.ssl-images-amazon.com/images...,58046,9.26,"Genre for Featured Categories,Drama",4.904762,0.633446,8.988378,21
36048,B0000A1QO4,Adventures In Odyssey: Fine Feathered Frenzy &...,Mike Joens,Dylan and Sal find themselves trapped in a wor...,NaN,229180,-1.00,"Independently Distributed,Kids & Family",4.954545,0.602042,8.959650,22
152327,B00DPUB5LU,Best of Collection: Family Affair,Brian Keith,From The Creator of My Three Sons Comes Anothe...,NaN,90305,9.54,"Genre for Featured Categories,Kids & Family",5.000000,0.583028,8.957569,20


In [28]:
product_df.tail()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
21656,B00000IBQH,"'N Sync, Never Enough VHS",NaN,music,NaN,832207,1.72,"Genre for Featured Categories,Music Videos & C...",1.181818,-0.129576,3.357878,22
71533,B000N2HBIY,Diary of a Cannibal,Trevor Parsons,INSPIRED BY A REAL-LIFE CASE OF A WOMAN WHO MU...,NaN,315320,14.98,"Studio Specials,Lionsgate Home Entertainment,A...",1.200000,-0.225561,3.136098,35
179310,B01C7RBFNE,Just Our Luck,Sarah Jayne Rothkopf,"After opening a mysterious fortune cookie, wit...",NaN,149387,14.98,"Independently Distributed,Comedy",1.344615,-0.305184,3.081655,325
81949,B000WZALY8,Juarez Mexico,James Cahill,Since 1993 more than 400 girls have been kille...,NaN,52787,6.32,"Genre for Featured Categories,Action & Adventure",1.095238,-0.260645,2.943626,21
133175,B005VU53SS,Murder at Monogram,Gil Gilbert,In 1945 Monogram pictures is about to start pr...,NaN,516713,24.95,"Genre for Featured Categories,Action & Adventure",1.027027,-0.246286,2.911311,37


In [45]:
best_products_stat = []

with tqdm(total=PRODUCT_SAMPLE_N) as pbar:
    for i in range(PRODUCT_SAMPLE_N):
        product = product_df.iloc[i]
        # choose longest reviews
        comments = comment_df[comment_df['asin'] == product['asin']]
        comments = comments.sort_values(by=['wordCount'], ascending=False)
        best_products_stat.append({
            'rank': i + 1,
            'samples': parse_joined_comments_to_json(comments.head(COMMENT_SAMPLE_N))
        })
        pbar.update(1)

100%|██████████| 5/5 [02:23<00:00, 28.73s/it]


In [46]:
with open(os.path.join(output_base_dir, 'best_products.json'), 'w') as f:
    json.dump(best_products_stat, f, cls=NpEncoder)

## Worst Products

In [47]:
product_df = product_df.sort_values(by=['avgSentimentWithRating'], ascending=True)

In [50]:
product_df.head()

,asin,title,brand,description,imageUrl,rank,price,categories,avgOverall,avgSentiment,avgSentimentWithRating,reviewCount
133175,B005VU53SS,Murder at Monogram,Gil Gilbert,In 1945 Monogram pictures is about to start pr...,NaN,516713,24.95,"Genre for Featured Categories,Action & Adventure",1.027027,-0.246286,2.911311,37
81949,B000WZALY8,Juarez Mexico,James Cahill,Since 1993 more than 400 girls have been kille...,NaN,52787,6.32,"Genre for Featured Categories,Action & Adventure",1.095238,-0.260645,2.943626,21
179310,B01C7RBFNE,Just Our Luck,Sarah Jayne Rothkopf,"After opening a mysterious fortune cookie, wit...",NaN,149387,14.98,"Independently Distributed,Comedy",1.344615,-0.305184,3.081655,325
71533,B000N2HBIY,Diary of a Cannibal,Trevor Parsons,INSPIRED BY A REAL-LIFE CASE OF A WOMAN WHO MU...,NaN,315320,14.98,"Studio Specials,Lionsgate Home Entertainment,A...",1.200000,-0.225561,3.136098,35
21656,B00000IBQH,"'N Sync, Never Enough VHS",NaN,music,NaN,832207,1.72,"Genre for Featured Categories,Music Videos & C...",1.181818,-0.129576,3.357878,22


In [51]:
worst_products_stat = []

with tqdm(total=PRODUCT_SAMPLE_N) as pbar:
    for i in range(PRODUCT_SAMPLE_N):
        product = product_df.iloc[i]
        comments = comment_df[comment_df['asin'] == product['asin']]
        comments = comments.sort_values(by=['wordCount'], ascending=False)
        worst_products_stat.append({
            'rank': i + 1,
            'samples': parse_joined_comments_to_json(comments.head(COMMENT_SAMPLE_N))
        })
        pbar.update(1)

100%|██████████| 5/5 [02:23<00:00, 28.76s/it]


In [52]:
with open(os.path.join(output_base_dir, 'worst_products.json'), 'w') as f:
    json.dump(worst_products_stat, f, cls=NpEncoder)

: 